In [1]:
class MNISTLoader():
    """数据获取及预处理 tf.keras.datasets"""
    def __init__(self):
        mnist = tf.keras.datasets.mnist
        (self.train_data, self.train_label), (self.test_data, self.test_label) = mnist.load_data()
        # MNIST中的图像默认为uint8（0-255的数字）。以下代码将其归一化到0-1之间的浮点数，并在最后增加一维作为颜色通道
        self.train_data = np.expand_dims(self.train_data.astype(np.float32) / 255.0, axis=-1)  # [60000, 28, 28, 1]
        self.test_data = np.expand_dims(self.test_data.astype(np.float32) / 255.0, axis=-1) # [10000, 28, 28, 1]
        self.train_label = self.train_label.astype(np.int32)    # [60000]
        self.test_label = self.test_label.astype(np.int32)      # [10000]
        self.num_train_data, self.num_test_data = self.train_data.shape[0], self.test_data.shape[0]
    
    def get_batch(self, batch_size):
        # 从数据集中随机取出batch_size个元素并返回
        index = np.random.randint(0, self.num_train_data, batch_size)
        return  self.train_data[index, :], self.train_label[index]

In [2]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()   # tensorflow v2 使用 v1 的语句
import numpy as np
import os

"""配置GPU分配"""
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1"  #选择哪一块gpu,如果是-1，就是调用cpu
config = tf.ConfigProto() #对session进行参数配置
config.allow_soft_placement=True #如果你指定的设备不存在，允许TF自动分配设备
config.gpu_options.per_process_gpu_memory_fraction=0.8 #分配百分之七十的显存给程序使用，避免内存溢出，可以自己调整
config.gpu_options.allow_growth = True #按需分配显存，这个比较重要

class MLP(tf.keras.Model):
    """模型的构建: tf.keras.model, tf.keras.layers"""
    def __init__(self):
        super().__init__()
        self.flatten = tf.keras.layers.Flatten # Flatten层将除第一维（batch_size）以外的维度展平
        self.dense1 = tf.keras.layers.Dense(units=100, activation=tf.nn.relu)
        self.dense2 = tf.keras.layers.Dense(units=10)
    
    def call(self, inputs): # [batch_size, 28, 28, 1]
        x = self.flatten()(inputs) #[batch_size, 28*28*1]
        x = self.dense1(x) #[batch_size, 100(units)]
        x = self.dense2(x) #[batch_size, 10(units)]
        output = tf.nn.softmax(x)
        return output

Instructions for updating:
non-resource variables are not supported in the long term


In [3]:
"""模型的训练: tf.keras.losses, tf.keras.optimizer"""

tf.enable_eager_execution()

"""定义一些模型超参数"""
num_epochs = 5
batch_size = 50
learning_rate = 0.001

"""实例化模型和数据读取类，并实例化一个 tf.keras.optimizer 的优化器（这里使用常用的 Adam 优化器）"""
model = MLP()
data_loader = MNISTLoader()
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

# 然后迭代进行以下步骤：
# 1. 从 DataLoader 中随机取一批训练数据；
# 2. 将这批数据送入模型，计算出模型的预测值；
# 3. 将模型预测值与真实值进行比较，计算损失函数（loss）。这里使用 tf.keras.losses 中的交叉熵函数作为损失函数；
# 4. 计算损失函数关于模型变量的导数；
# 5. 将求出的导数值传入优化器，使用优化器的 apply_gradients 方法更新模型参数以最小化损失函数。

num_batches = int(data_loader.num_train_data // batch_size * num_epochs)
for batch_index in range(num_batches):
    X, y = data_loader.get_batch(batch_size)
    with tf.GradientTape() as tape:
        y_pred = model(X)
        loss = tf.keras.losses.sparse_categorical_crossentropy(y_true=y, y_pred=y_pred)
        loss = tf.reduce_mean(loss)
        print("batch %d: loss %f" % (batch_index, loss.numpy()))
    grads = tape.gradient(loss, model.variables)
    optimizer.apply_gradients(grads_and_vars=zip(grads, model.variables))

batch 0: loss 2.300875
batch 1: loss 2.445652
batch 2: loss 2.230669
batch 3: loss 2.228269
batch 4: loss 2.009769
batch 5: loss 2.090004
batch 6: loss 1.970027
batch 7: loss 1.953643
batch 8: loss 1.824839
batch 9: loss 1.628621
batch 10: loss 1.728665
batch 11: loss 1.549103
batch 12: loss 1.557637
batch 13: loss 1.654827
batch 14: loss 1.615896
batch 15: loss 1.423028
batch 16: loss 1.313239
batch 17: loss 1.319187
batch 18: loss 1.189416
batch 19: loss 1.364369
batch 20: loss 1.358119
batch 21: loss 1.135534
batch 22: loss 1.071933
batch 23: loss 1.183190
batch 24: loss 1.068400
batch 25: loss 1.083597
batch 26: loss 1.028986
batch 27: loss 1.049774
batch 28: loss 1.117654
batch 29: loss 0.898158
batch 30: loss 0.737146
batch 31: loss 0.987828
batch 32: loss 0.803879
batch 33: loss 0.769617
batch 34: loss 0.800383
batch 35: loss 0.775818
batch 36: loss 0.794675
batch 37: loss 0.771361
batch 38: loss 0.909528
batch 39: loss 0.821828
batch 40: loss 0.668118
batch 41: loss 0.847236
ba

batch 369: loss 0.562011
batch 370: loss 0.282438
batch 371: loss 0.330038
batch 372: loss 0.192429
batch 373: loss 0.135018
batch 374: loss 0.133391
batch 375: loss 0.284957
batch 376: loss 0.353694
batch 377: loss 0.200396
batch 378: loss 0.319815
batch 379: loss 0.313637
batch 380: loss 0.127590
batch 381: loss 0.348419
batch 382: loss 0.264775
batch 383: loss 0.183145
batch 384: loss 0.266699
batch 385: loss 0.284903
batch 386: loss 0.154354
batch 387: loss 0.178581
batch 388: loss 0.473145
batch 389: loss 0.324855
batch 390: loss 0.157189
batch 391: loss 0.184297
batch 392: loss 0.267168
batch 393: loss 0.480851
batch 394: loss 0.280595
batch 395: loss 0.557021
batch 396: loss 0.251989
batch 397: loss 0.290169
batch 398: loss 0.216938
batch 399: loss 0.314786
batch 400: loss 0.249774
batch 401: loss 0.484825
batch 402: loss 0.265161
batch 403: loss 0.410580
batch 404: loss 0.161680
batch 405: loss 0.245510
batch 406: loss 0.350581
batch 407: loss 0.190694
batch 408: loss 0.385686


batch 702: loss 0.154230
batch 703: loss 0.131111
batch 704: loss 0.228476
batch 705: loss 0.098760
batch 706: loss 0.229227
batch 707: loss 0.235259
batch 708: loss 0.265474
batch 709: loss 0.386338
batch 710: loss 0.121830
batch 711: loss 0.240153
batch 712: loss 0.124217
batch 713: loss 0.178742
batch 714: loss 0.070161
batch 715: loss 0.143694
batch 716: loss 0.219581
batch 717: loss 0.275227
batch 718: loss 0.097224
batch 719: loss 0.237399
batch 720: loss 0.330999
batch 721: loss 0.212000
batch 722: loss 0.159874
batch 723: loss 0.243831
batch 724: loss 0.244349
batch 725: loss 0.264588
batch 726: loss 0.087605
batch 727: loss 0.254331
batch 728: loss 0.039929
batch 729: loss 0.060395
batch 730: loss 0.287774
batch 731: loss 0.323973
batch 732: loss 0.253084
batch 733: loss 0.176079
batch 734: loss 0.241553
batch 735: loss 0.144099
batch 736: loss 0.177530
batch 737: loss 0.115648
batch 738: loss 0.222471
batch 739: loss 0.369958
batch 740: loss 0.175455
batch 741: loss 0.097437


batch 1036: loss 0.248928
batch 1037: loss 0.217424
batch 1038: loss 0.248170
batch 1039: loss 0.155171
batch 1040: loss 0.380582
batch 1041: loss 0.113283
batch 1042: loss 0.076463
batch 1043: loss 0.084851
batch 1044: loss 0.136976
batch 1045: loss 0.138654
batch 1046: loss 0.083772
batch 1047: loss 0.172815
batch 1048: loss 0.125235
batch 1049: loss 0.141084
batch 1050: loss 0.372623
batch 1051: loss 0.117978
batch 1052: loss 0.142924
batch 1053: loss 0.052428
batch 1054: loss 0.274194
batch 1055: loss 0.207536
batch 1056: loss 0.152674
batch 1057: loss 0.127353
batch 1058: loss 0.105676
batch 1059: loss 0.119079
batch 1060: loss 0.160755
batch 1061: loss 0.095152
batch 1062: loss 0.052932
batch 1063: loss 0.093468
batch 1064: loss 0.146355
batch 1065: loss 0.050166
batch 1066: loss 0.317372
batch 1067: loss 0.054155
batch 1068: loss 0.086294
batch 1069: loss 0.130341
batch 1070: loss 0.059564
batch 1071: loss 0.127917
batch 1072: loss 0.202305
batch 1073: loss 0.149619
batch 1074: 

batch 1367: loss 0.270687
batch 1368: loss 0.185180
batch 1369: loss 0.288889
batch 1370: loss 0.083915
batch 1371: loss 0.083030
batch 1372: loss 0.149212
batch 1373: loss 0.122481
batch 1374: loss 0.130495
batch 1375: loss 0.120040
batch 1376: loss 0.221912
batch 1377: loss 0.194405
batch 1378: loss 0.201421
batch 1379: loss 0.164939
batch 1380: loss 0.104175
batch 1381: loss 0.204325
batch 1382: loss 0.289109
batch 1383: loss 0.075465
batch 1384: loss 0.113669
batch 1385: loss 0.111186
batch 1386: loss 0.070071
batch 1387: loss 0.181922
batch 1388: loss 0.363858
batch 1389: loss 0.091408
batch 1390: loss 0.269526
batch 1391: loss 0.075366
batch 1392: loss 0.115688
batch 1393: loss 0.149987
batch 1394: loss 0.133172
batch 1395: loss 0.198633
batch 1396: loss 0.179500
batch 1397: loss 0.167943
batch 1398: loss 0.032476
batch 1399: loss 0.104269
batch 1400: loss 0.069466
batch 1401: loss 0.090353
batch 1402: loss 0.292432
batch 1403: loss 0.222978
batch 1404: loss 0.098667
batch 1405: 

...

batch 4656: loss 0.210203
batch 4657: loss 0.072731
batch 4658: loss 0.072908
batch 4659: loss 0.083795
batch 4660: loss 0.016079
batch 4661: loss 0.133848
batch 4662: loss 0.102517
batch 4663: loss 0.043878
batch 4664: loss 0.062752
batch 4665: loss 0.017648
batch 4666: loss 0.057854
batch 4667: loss 0.054852
batch 4668: loss 0.037323
batch 4669: loss 0.045335
batch 4670: loss 0.084480
batch 4671: loss 0.041495
batch 4672: loss 0.035407
batch 4673: loss 0.026118
batch 4674: loss 0.107618
batch 4675: loss 0.035290
batch 4676: loss 0.025204
batch 4677: loss 0.033538
batch 4678: loss 0.020705
batch 4679: loss 0.027136
batch 4680: loss 0.029902
batch 4681: loss 0.022100
batch 4682: loss 0.044887
batch 4683: loss 0.031861
batch 4684: loss 0.019901
batch 4685: loss 0.029349
batch 4686: loss 0.043696
batch 4687: loss 0.066684
batch 4688: loss 0.074131
batch 4689: loss 0.059195
batch 4690: loss 0.062882
batch 4691: loss 0.047730
batch 4692: loss 0.055117
batch 4693: loss 0.014056
batch 4694: 

batch 4990: loss 0.063442
batch 4991: loss 0.019226
batch 4992: loss 0.035609
batch 4993: loss 0.049650
batch 4994: loss 0.077249
batch 4995: loss 0.090902
batch 4996: loss 0.015036
batch 4997: loss 0.038061
batch 4998: loss 0.039662
batch 4999: loss 0.047229
batch 5000: loss 0.031925
batch 5001: loss 0.027703
batch 5002: loss 0.061898
batch 5003: loss 0.079779
batch 5004: loss 0.033264
batch 5005: loss 0.022491
batch 5006: loss 0.089823
batch 5007: loss 0.054719
batch 5008: loss 0.050034
batch 5009: loss 0.065279
batch 5010: loss 0.033134
batch 5011: loss 0.098903
batch 5012: loss 0.155036
batch 5013: loss 0.051168
batch 5014: loss 0.044356
batch 5015: loss 0.075649
batch 5016: loss 0.021787
batch 5017: loss 0.140857
batch 5018: loss 0.014794
batch 5019: loss 0.290651
batch 5020: loss 0.123114
batch 5021: loss 0.008831
batch 5022: loss 0.066160
batch 5023: loss 0.044592
batch 5024: loss 0.047501
batch 5025: loss 0.181251
batch 5026: loss 0.051630
batch 5027: loss 0.035827
batch 5028: 

batch 5322: loss 0.017175
batch 5323: loss 0.037624
batch 5324: loss 0.086561
batch 5325: loss 0.038721
batch 5326: loss 0.011830
batch 5327: loss 0.005231
batch 5328: loss 0.024659
batch 5329: loss 0.010957
batch 5330: loss 0.034327
batch 5331: loss 0.032976
batch 5332: loss 0.013842
batch 5333: loss 0.020804
batch 5334: loss 0.392538
batch 5335: loss 0.121329
batch 5336: loss 0.075269
batch 5337: loss 0.035815
batch 5338: loss 0.060007
batch 5339: loss 0.043003
batch 5340: loss 0.019024
batch 5341: loss 0.309374
batch 5342: loss 0.037797
batch 5343: loss 0.011164
batch 5344: loss 0.040227
batch 5345: loss 0.062091
batch 5346: loss 0.022448
batch 5347: loss 0.057218
batch 5348: loss 0.046942
batch 5349: loss 0.044432
batch 5350: loss 0.064022
batch 5351: loss 0.073774
batch 5352: loss 0.012096
batch 5353: loss 0.017706
batch 5354: loss 0.126077
batch 5355: loss 0.008459
batch 5356: loss 0.084962
batch 5357: loss 0.059706
batch 5358: loss 0.026083
batch 5359: loss 0.174412
batch 5360: 

batch 5653: loss 0.085384
batch 5654: loss 0.143165
batch 5655: loss 0.308290
batch 5656: loss 0.040000
batch 5657: loss 0.010401
batch 5658: loss 0.035616
batch 5659: loss 0.009828
batch 5660: loss 0.024764
batch 5661: loss 0.017502
batch 5662: loss 0.010494
batch 5663: loss 0.007100
batch 5664: loss 0.041112
batch 5665: loss 0.024268
batch 5666: loss 0.017316
batch 5667: loss 0.045041
batch 5668: loss 0.031783
batch 5669: loss 0.061132
batch 5670: loss 0.004389
batch 5671: loss 0.034980
batch 5672: loss 0.034200
batch 5673: loss 0.041014
batch 5674: loss 0.055587
batch 5675: loss 0.016107
batch 5676: loss 0.044787
batch 5677: loss 0.013151
batch 5678: loss 0.008952
batch 5679: loss 0.018778
batch 5680: loss 0.146369
batch 5681: loss 0.047101
batch 5682: loss 0.060601
batch 5683: loss 0.017994
batch 5684: loss 0.027624
batch 5685: loss 0.083301
batch 5686: loss 0.018357
batch 5687: loss 0.019415
batch 5688: loss 0.066686
batch 5689: loss 0.005485
batch 5690: loss 0.016175
batch 5691: 

batch 5988: loss 0.037204
batch 5989: loss 0.028066
batch 5990: loss 0.063529
batch 5991: loss 0.120459
batch 5992: loss 0.008769
batch 5993: loss 0.010463
batch 5994: loss 0.023318
batch 5995: loss 0.112096
batch 5996: loss 0.047481
batch 5997: loss 0.270669
batch 5998: loss 0.083184
batch 5999: loss 0.012994


In [4]:
"""模型的评估：tf.keras.metrics"""
sparse_categorical_accuracy = tf.keras.metrics.SparseCategoricalAccuracy()
num_batches = int(data_loader.num_test_data // batch_size)
for batch_index in range(num_batches):
    start_index, end_index = batch_index * batch_size, (batch_index + 1) * batch_size
    y_pred = model.predict(data_loader.test_data[start_index: end_index])
    sparse_categorical_accuracy.update_state(y_true=data_loader.test_label[start_index: end_index], y_pred=y_pred)
print("test accuracy: %f" % sparse_categorical_accuracy.result())

test accuracy: 0.975600
